Datasets:
Complete Binary, w/ background - sizes 1 and 10 and highly mixed 5
Complete Binary, no background - sizes 1 and 10 and incremental 5
Custom Smoothing, w/ background - sizes 1 and 10 - This is the baseline
Custom Smoothing, no background - sizes 1 and 10


Loss settings:
Custom Smoothing, enabled - Loss A -This is the baseline
Custom Smoothing, disabled - Loss A
No Smoothing, disabled - Loss A or B 
Proper Smoothing, disabled - Loss B


Model Types: 
Standard Dilation, No Attention - This is the baseline
Standard Dilation, Attention
Reduced Dilation, No Attention
Reduced Dilation, Attention

Todo: Need to make a reduced dilation version of the model - done, model takes dilation scaler
Confirm that if no smoothing and early reward disabled, Loss A and Loss B are equivalent - confirmed, so long as parameters and thresholds are the same

Checklist of datasets:

 Complete Binary, w/ background

 Size 1 - COMPLETE

 Size 10

 Highly mixed 5


 Complete Binary, no background

 Size 1

 Size 10

 Incremental 5 - have a custom smoothed version w/ background that can be converted

 Custom Smoothing, w/ background (This is the baseline)

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Custom Smoothing, no background

 Size 1

 Size 10


In [2]:
import time
import sys
import os
import glob
import math
import threading
import concurrent.futures as cf
import random
import re

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
from keras import backend as K
import gc
import keras_tuner as kt
from pyfaidx import Fasta

import sys
import os

datasets_path = "../../Datasets/"
models_path = "../../Models/"

2025-04-12 21:51:14.932147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-12 21:51:15.028668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-12 21:51:15.056663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-12 21:51:15.232283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-12 21:51:16.827781: W tensorflow/compiler/tf2

In [3]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
import IEModules as iem

In [6]:
iem.Genetic_Data_Pipeline.test_function("Howdy!")

You entered Howdy! into test_function


'Howdy!'

In [8]:
import inspect
functions = inspect.getmembers(iem.Genetic_Data_Pipeline, inspect.isfunction)
for name, func in functions:
    print(name)

Input
_bytes_feature
_float_feature
_int64_feature
build_chunk_data_for_indices
build_dataset_from_tfrecords
bytes_feature
calculate_introns
compute_chunk_indices
convert_and_write_tfrecord
convert_labels_to_binary
float_feature_list
int_feature_list
label_sequence_local
load_gtf_annotations
main
one_hot_encode_reference
pad_encoded_seq
pad_labels
parse_chunk_example
search_gtf_by_range
serialize_chunk_example
serialize_example_with_metadata_no_convert
split_tfrecords
stream_shuffled_records
swap_columns_if_needed
test_dataset_from_tfrecords
test_function
trim_chr_genome
tvt_split_tfrecords
write_shuffled_records_to_single_tfrecord
write_tfrecord_in_shards_hybrid
write_to_shard_with_threads


Check Genetic_Data_Pipeline for correctness

Check Custom_Models for correctness

Need to make a shard shuffler module probably that goes from output of genetic_data_pipeline to fully shuffled test-train-split

In [3]:
iem.Genetic_Data_Pipeline.dataset_pipeline_help()


          1. build_initial_shards function generates 4 tfrecord shards at each shift fed to it.
          I recommend giving a list of 1 window shift.  Currently paths are hardcoded.  
          The first 4*n characters of the shards will be the shift numbers where n is 
          the number of shifts fed to the function.
          
          2. split_tfrecords takes an input and output directory and number of file splits
          and splits each TFRecord in the input_directory (assumed to be gzipped TFRecords)
          into 'num_splits' smaller TFRecords.
          
          3. write_shuffled_records_to_single_tfrecord builds a single big, shuffled, 
          gzip-compressed TFRecord file 
          
          Args:
            input_dir (str): Directory containing the source TFRecord files.
            allowed_indices (list): List of allowed starting indices.
            output_filepath (str): Full path to the output TFRecord file.
            
          4. tvt_split_records tes

Checklist of datasets:

 Complete Binary, w/ background

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Highly mixed 5                                 Need to build highly mixed 5 and convert to binary


 Complete Binary, no background

 Size 1 - background removed when loading into model (Data_Functions.py)

 Size 10 - background removed when loading into model (Data_Functions.py)

 Incremental 5 - have a custom smoothed version w/ background that can be converted   conversion ongoing

 Custom Smoothing, w/ background (This is the baseline)

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Custom Smoothing, no background

 Size 1 - background removed when loading into model (Data_Functions.py)

 Size 10 - background removed when loading into model (Data_Functions.py)



 Size 10 has 2365261 records

In [ ]:
# iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/all_ten_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_ten_shuffled.tfrecord.gz")

I0000 00:00:1744095490.509284 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.699680 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.699799 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.704425 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.704488 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [ ]:
# iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/fifths_incremental_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_fifths_incrememntal.tfrecord.gz")

In [ ]:
# iem.Genetic_Data_Pipeline.transform_tfdataset(datasets_path+"AugDataSets/AllTen", datasets_path+"AugDataSets", 45, 0.5)
# print("Done")

Temporary folder created at: ../../Datasets/AugDataSets/AllTen/temp_shards


I0000 00:00:1744246104.885834  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.075605  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.075745  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.078746  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.078828  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Total records found in input dataset: 2365261
Sampling 1182630 records (fraction = 0.5).


2025-04-09 19:14:36.772563: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Split all_ten_shuffled.tfrecord.gz into 45 shards.
10000 records written so far...
20000 records written so far...
30000 records written so far...
40000 records written so far...
50000 records written so far...
60000 records written so far...
70000 records written so far...
80000 records written so far...
90000 records written so far...
100000 records written so far...
110000 records written so far...
120000 records written so far...
130000 records written so far...
140000 records written so far...
150000 records written so far...
160000 records written so far...
170000 records written so far...
180000 records written so far...
190000 records written so far...
200000 records written so far...
210000 records written so far...
220000 records written so far...
230000 records written so far...
240000 records written so far...
250000 records written so far...
260000 records written so far...
270000 records written so far...
280000 records written so far...
290000 records written so far...
3

In [ ]:
# iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/50_all_ten_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_5x_mixed.tfrecord.gz")

2025-04-10 04:28:15.601091: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Dataset New Naming Scheme: 
Epoch Units: 01-10, Style: M for Mixed or I for Incremental, Smoothing: C for custom smoothing, B for Binary
Epoch-Units_Style_Smoothing_list-of-shifts-if-incremental-divided-by-1000_IEData.tfrecord.gz
ex: 02_I_B_0-2.5_IEData.tfrecord.gz

Per line, this is input from trial csv:
Trial,Smoothing,Background,Early Rewarding,Size,Style,Dilation,Attention

Trial becomes folder where model data are saved

Smoothing, Size, Style define the dataset

Background is a choice handled in the data loader which returns a dataset generator pretending to be a dataset (Data_Functions.prep_dataset_from_tfrecord())

Early rewarding: if true, need an option in the loss functions or something that checks epoch number.  Rewarding==True for more than 3 epoch units is overkill so the switch should flip after 3 epoch units

Dilation and Attention are model hyperparameters

What I need then:
Experiment handler function (class?) that reads a line from the csv and trains for 10 epoch units.  Needs a working folder given to it (Experiment Name is probably the folder name) and a subfolder for each trial (Trial_01, etc).  Checkpoints get saved in there, as does the model history which gets saved to json every epoch unit but overwritten, as should the StatefulReduceLROnPlateau callback state that I have yet to implement.
The experiment should track which trial and which epoch was most recently saved and be able to resume from that trial and checkpoint easily.  Maybe have it save a jpg of the train validate curve at the end of the trial using the function in Helper_functions.py

In [5]:
data_directory = datasets_path+"Experiment 01/"
save_directory = data_directory+"TestValTrain/"
for root, dirs, files in os.walk(data_directory):
    if 'TestValTrain' in dirs:
        dirs.remove('TestValTrain')
    for file in files:
        file_path = data_directory+os.path.basename(file)
        train_save_file_path = save_directory+"Train_"+os.path.basename(file)
        val_save_file_path = save_directory+"Val_"+os.path.basename(file)
        test_save_file_path = save_directory+"Test_"+os.path.basename(file)
        iem.Genetic_Data_Pipeline.tvt_split_tfrecords(file_path, train_save_file_path, val_save_file_path,test_save_file_path)
        print(os.path.basename(file)+" split")

I0000 00:00:1744516349.562100 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.758718 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.758835 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.762785 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.762879 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Total records found: 1182630
Splitting into -> Train: 946104, Val: 118263, Test: 118263


2025-04-12 21:58:53.921334: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 22613 of 25000
2025-04-12 21:58:55.008046: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2025-04-12 22:51:29.014481: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Final Split Counts -> Train: 946104, Val: 118263, Test: 118263
05_M_B_IEData.tfrecord.gz split
Total records found: 2365261
Splitting into -> Train: 1892208, Val: 236526, Test: 236527


2025-04-12 23:04:33.559500: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:15: Filling up shuffle buffer (this may take a while): 20358 of 25000
2025-04-12 23:04:35.240383: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


Final Split Counts -> Train: 1892208, Val: 236526, Test: 236527
10_M_C_IEData.tfrecord.gz split


2025-04-13 00:55:15.264840: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# #!/usr/bin/env python3
# """
# Experiment Handler Module

# This module defines an ExperimentHandler class that:
#   • Reads a single trial’s configuration (with keys: Trial, Smoothing, Background,
#     Early Rewarding, Size, Style, Dilation, Attention) from a CSV file.
#   • Creates an experiment folder under a common experiment root for each trial.
#   • Loads a dataset from a tfrecord using a constructed filename based on the configuration.
#   • Builds a model via a provided model builder or a default placeholder.
#   • Trains the model for 10 "epoch units" (one epoch per unit in this example).
#   • Saves checkpoints, training history (using Helper_Functions.save_history_to_json),
#     and a train/validation curve plot (using Helper_Functions.plot_train_val_curve) after each epoch unit.
#   • Supports resuming progress from a saved state.
# """


# import time
# import sys
# import os
# import glob
# import math
# import threading
# import concurrent.futures as cf
# import random
# import re
# import json

# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
# from keras import backend as K
# import gc
# import keras_tuner as kt
# from pyfaidx import Fasta

# import sys
# import os

# datasets_path = "../../Datasets/"
# models_path = "../../Models/"
# import IEModules as iem


# # Optionally import custom callbacks from Custom_Callbacks if available
# # from Custom_Callbacks import StatefulReduceLROnPlateau  # placeholder for stateful LR scheduling

# class ExperimentHandler:
#     def __init__(self, trial_config, experiment_root="ExperimentResults", model_builder=None):
#         """
#         Initialize experiment for a single trial.
        
#         Parameters:
#           trial_config (dict): Dictionary with keys:
#              "Trial", "Smoothing", "Background", "Early Rewarding", "Size", "Style", "Dilation", "Attention"
#           experiment_root (str): The root directory for experiment results.
#           model_builder (callable, optional): Function that builds and compiles the model.
#                Defaults to a simple placeholder model.
#         """
#         self.trial_config = trial_config
#         self.total_epoch_units = 10
#         self.current_epoch_unit = 0
#         self.model = None
#         self.model_builder = model_builder if model_builder is not None else self.default_model_builder

#         # Create a subfolder for the trial using the "Trial" field under the experiment root.
#         self.trial_folder = os.path.join(experiment_root, self.trial_config["Trial"])
#         os.makedirs(self.trial_folder, exist_ok=True)
#         self.state_file = os.path.join(self.trial_folder, "state.json")
#         self.dataset = None

#     def default_model_builder(self):
#         """
#         Build a default placeholder model.
#         Replace this with a call to your custom model builder (e.g., Custom_Models.create_dcnn_model).
#         """
#         model = models.Sequential([
#             layers.InputLayer(input_shape=(5000, 5)),
#             layers.Conv1D(32, 3, activation='relu', padding='same'),
#             layers.GlobalAveragePooling1D(),
#             layers.Dense(5, activation='sigmoid')
#         ])
#         model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["f1_score"])
#         return model

#     def load_dataset(self):
#         """
#         Constructs the tfrecord filename based on trial configuration and loads the dataset.
#         Modify the logic below to reflect your specific naming convention.
#         """
#         # For example: "Epoch-Units_Style_Smoothing_list-of-shifts-if-incremental-divided-by-1000_IEData.tfrecord.gz"
#         epoch_unit_placeholder = "XX"  # Placeholder: replace or update with epoch unit if needed.
#         style = self.trial_config["Style"]  # e.g., "M" or "I"
#         smoothing = self.trial_config["Smoothing"]  # e.g., "C" (custom) or "B" (binary)
#         # For incremental style, include shifts (placeholder here as "0-2.5")
#         shifts = "0-2.5" if style.upper() == "I" else ""
#         file_name = f"{epoch_unit_placeholder}_{style}_{smoothing}_{shifts}_IEData.tfrecord.gz"
#         # Adjust the base path for your tfrecords:
#         tfrecord_path = os.path.join("path_to_tfrecords", file_name)
        
#         # Determine if the Background should be removed (interpreting string to boolean).
#         cut_background = str(self.trial_config["Background"]).lower() in ["true", "1"]
#         # Use the "Size" field for the batch size.
#         batch_size = int(self.trial_config["Size"])
#         self.dataset = iem.Data_Functions.prep_dataset_from_tfrecord(
#             tfrecord_path,
#             batch_size=batch_size,
#             shuffled=True,
#             cut_background=cut_background
#         )
    
#     def resume_progress(self):
#         """
#         Resumes from a previously saved state if it exists.
#         Loads the current epoch unit and last saved checkpoint.
#         """
#         if os.path.exists(self.state_file):
#             with open(self.state_file, "r") as f:
#                 state = json.load(f)
#             self.current_epoch_unit = state.get("current_epoch_unit", 0)
#             last_checkpoint = state.get("last_checkpoint", None)
#             if last_checkpoint and os.path.exists(last_checkpoint):
#                 self.model = models.load_model(last_checkpoint, compile=False)
#             print(f"Resumed trial {self.trial_config['Trial']} at epoch unit {self.current_epoch_unit}")
#         else:
#             print(f"Starting new trial {self.trial_config['Trial']}")
    
#     def save_state(self, checkpoint_path):
#         """
#         Saves the current experiment state to allow resuming later.
#         """
#         state = {
#             "current_epoch_unit": self.current_epoch_unit,
#             "last_checkpoint": checkpoint_path
#         }
#         with open(self.state_file, "w") as f:
#             json.dump(state, f)
#         print(f"Saved state for trial {self.trial_config['Trial']} to {self.state_file}")

#     def train_epoch_unit(self):
#         """
#         Trains the model for one epoch unit.
#         Implements early rewarding logic (active only for the first 3 epoch units),
#         saves a checkpoint, training history, and a plot of the train/validation curve.
#         """
#         checkpoint_path = os.path.join(self.trial_folder, f"checkpoint_epochunit_{self.current_epoch_unit+1}.h5")
#         cp_callback = callbacks.ModelCheckpoint(
#             filepath=checkpoint_path,
#             save_weights_only=False,
#             save_freq="epoch",
#             verbose=1
#         )
#         callbacks_list = [cp_callback]
        
#         # Early Rewarding: Only effective if set in the config and for the first 3 epoch units.
#         early_reward = str(self.trial_config["Early Rewarding"]).lower() in ["true", "1"]
#         if early_reward and self.current_epoch_unit >= 3:
#             early_reward = False
#         print(f"Trial {self.trial_config['Trial']}: Early Rewarding set to {early_reward} at epoch unit {self.current_epoch_unit+1}")
        
#         # Train for one epoch unit. (Here, one epoch = one epoch unit.)
#         history = self.model.fit(
#             self.dataset,
#             epochs=1,
#             callbacks=callbacks_list,
#             validation_data=self.dataset,  # Adjust for a dedicated validation dataset if available.
#             verbose=1
#         )
        
#         # Save the training history to JSON
#         iem.Helper_Functions.save_history_to_json(history, metadata=f"epochunit_{self.current_epoch_unit+1}", save_path=self.trial_folder)
        
#         # Plot and save the training and validation curve
#         fig = iem.Helper_Functions.plot_train_val_curve(history, training_target_variable="loss")  # Modify target variable as needed.
#         plot_path = os.path.join(self.trial_folder, f"train_val_curve_epochunit_{self.current_epoch_unit+1}.jpg")
#         fig.savefig(plot_path)
#         plt.close(fig)
        
#         self.current_epoch_unit += 1
#         self.save_state(checkpoint_path)
    
#     def run_trial(self):
#         """
#         Runs the trial by loading the dataset, initializing (or resuming) the model,
#         and training for 10 epoch units.
#         """
#         self.load_dataset()
#         if self.model is None:
#             self.model = self.model_builder()
#         self.resume_progress()
#         while self.current_epoch_unit < self.total_epoch_units:
#             print(f"Trial {self.trial_config['Trial']}: Starting epoch unit {self.current_epoch_unit+1}/{self.total_epoch_units}")
#             self.train_epoch_unit()
#         print(f"Trial {self.trial_config['Trial']} completed.")

# def run_trials_from_csv(csv_path):
#     """
#     Reads the CSV file containing trial configurations and runs the experiment for each trial.
    
#     CSV file is expected to have the following columns:
#       Trial, Smoothing, Background, Early Rewarding, Size, Style, Dilation, Attention
#     """
#     df = pd.read_csv(csv_path)
    
#     # Define a root directory to hold all trial folders.
#     experiment_root = "ExperimentResults"
#     os.makedirs(experiment_root, exist_ok=True)
    
#     # Loop over each row (trial) and run the experiment.
#     for idx, row in df.iterrows():
#         trial_config = row.to_dict()
#         print(f"Starting trial: {trial_config.get('Trial', 'Unknown')}")
#         trial_handler = ExperimentHandler(trial_config, experiment_root=experiment_root)
#         trial_handler.run_trial()

# if __name__ == "__main__":
#     # Update this with the path to your CSV file containing trial configurations.
#     csv_path = "trial_config.csv"
#     run_trials_from_csv(csv_path)

┌────────────────────────────────────────────────────────────────────────┐
│  User/CLI                                                             │
│  $ python experiment_framework.py spec.csv EXP_NAME                  │
└────────────┬──────────────────────────────────────────────────────────┘
             │
             ▼
┌────────────────────────────────────────────────────────────────────────┐
│  ExperimentHandler(csv_path, experiment_name, …)                      │
│  ├─ reads CSV into DataFrame                                          │
│  └─ creates   <MODEL_DIR>/<EXP_NAME>/                                 │
└────────────┬──────────────────────────────────────────────────────────┘
             │
             ▼
┌────────────────────────────────────────────────────────────────────────┐
│  run()                                                                │
│  loop rows ▸ trial_row                                                │
│     │                                                                 │
│     ├─ _prepare_trial_folder()    →  …/Trial_XX/ (+ checkpoints/)     │
│     ├─ discover latest checkpoint ─┐                                  │
│     │                             └─▶  models.load_model()   │
│     └─ if none → _build_model()   →  create_modular_dcnn_model()      │
│                                                                       │
│     loop epoch-unit 0-9                                               │
│        ├─ _load_datasets()        →  Data_Functions.prep_dataset…     │
│        ├─ _make_callbacks()       →  {ModelCheckpoint, LR, TB…}       │
│        └─ model.fit(epochs=1)     →  train 1 *epoch-unit*             │
│               └─ Helper_Functions.save_history_to_json() (overwrite)  │
│                                                                       │
│     after 10 units                                                    │
│        └─ _finalise_trial()                                           │
│            ├─ plot_train_val_curve() → Trial_XX/train_val_curve.jpg   │
│            └─ model.save("final_model.keras")                         │
└────────────────────────────────────────────────────────────────────────┘


In [ ]:
# experiment_framework.py
"""
High-level experiment orchestration for the Intron-Exon project.

* 1 CSV == 1 experiment folder ⤏ each row is a *trial* that runs **10 epoch-units**.
* Handles automatic folder creation, checkpointing, history/curve saving, and resume logic.
* Designed to work with the existing IEModules package (Data_Functions, Custom_Models, etc.).

The implementation purposefully leaves a handful of 🔲 *fill-in-the-blank* areas where deeper
project knowledge is needed.  Search for the keyword `TODO` to complete the details.
"""

from __future__ import annotations

import os
import re
import json
import glob
import shutil
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import pandas as pd
import tensorflow as tf

# ── In-house modules ───────────────────────────────────────────────────────────
from IEModules import (
    Data_Functions,
    Custom_Models,
    Custom_Callbacks,
    Custom_Losses,
    Helper_Functions,
    Custom_Metrics
)
from IEModules.config import (
    DATA_DIR, 
    MODEL_DIR, 
    LOG_DIR, 
    EPOCH_UNIT_SIZE, 
    EPOCH_UNITS_PER_TRIAL, 
    DEFAULT_BATCH_SIZE, 
    STEPS_PER_EPOCH_UNIT, 
    experiment_data_folder,
    experiment_folder,
    LR_STATE_SAVE_SUBDIR,
    LR_STATE_SAVE_FILENAME,
    DOMINANT_CLASS_INDEX,
    DOMINANT_CORRECT_MULTIPLIER,
    DOMINANT_INCORRECT_MULTIPLIER,
    OTHER_TP_MULTIPLIER,
    OTHER_FN_MULTIPLIER,
    OTHER_FP_MULTIPLIER,
    OTHER_TN_MULTIPLIER,
    THRESHOLD,
    FOCAL_GAMMA,
    FOCAL_ALPHA,
    CORRECT_SMOOTHING_MULTIPLIER,
    INCORRECT_SMOOTHING_MULTIPLIER,
    DEFAULT_SMOOTHING_AS_CORRECT,
    LABEL_SMOOTHING,
    SWAP_EPOCH,
)



# Dataset naming helpers -------------------------------------------------------

def build_dataset_filepath(
    split: str,              # 'train' | 'val' | 'test'
    units: int,              # 1-10  (will be zero-padded to 02)
    style_flag: str,         # 'M' or 'I'
    smoothing_flag: str,     # 'C' or 'B'
    base_data_dir: str | Path = experiment_data_folder # Modify in config
) -> str:
    """
    Return the *full* path to the TFRecord whose filename starts with
    '{split}_{units:02d}_{style_flag}_{smoothing_flag}' (case-insensitive).

    Raises:
        FileNotFoundError - no matching file in base_data_dir
        RuntimeError      - more than one match (ambiguous)
    """
    split   = split.lower()
    prefix  = f"{split}_{units:02d}_{style_flag.upper()}_{smoothing_flag.upper()}"
    pattern = f"{prefix}*IEData.tfrecord.gz"        # wildcard swallows shift part
    matches = glob.glob(str(Path(base_data_dir) / pattern), recursive=False)

    if not matches:
        raise FileNotFoundError(f"No TFRecord found for prefix '{prefix}' in {base_data_dir}")
    if len(matches) > 1:
        raise RuntimeError(f"Ambiguous prefix '{prefix}': {matches}")

    return matches[0]   # the unique match



# ── Utility functions ─────────────────────────────────────────────────────────

def latest_checkpoint(ckpt_dir: Path) -> Optional[Path]:
    """Return the most recent .keras checkpoint inside *ckpt_dir* (None if absent)."""
    checkpoints = sorted(ckpt_dir.glob("*.keras"))
    return checkpoints[-1] if checkpoints else None


def parse_epoch_from_ckpt(ckpt_path: Path) -> int:
    """Extract integer epoch-unit from a filename like 'epoch-05-val_metric-0.1234.keras'."""
    m = re.search(r"epoch[-_](\d+)", ckpt_path.name)
    return int(m.group(1)) if m else 0


# ── Main class ────────────────────────────────────────────────────────────────

class ExperimentHandler:
    """Drive the end-to-end training process described by a CSV file."""

    def __init__(
        self,
        csv_path: Path | str, # The csv with trial parameters
        experiment_name: str,
        model_dir: Path | str = MODEL_DIR,
        data_folder: Path | str = experiment_data_folder,  
        resume: bool = True,
        batch_size: int = DEFAULT_BATCH_SIZE,
    ) -> None:
        self.csv_path      = Path(csv_path)
        self.experiment    = experiment_name
        self.batch_size    = batch_size
        self.resume_flag   = resume
        self.experiment_dir = Path(model_dir) / experiment_name
        self.experiment_dir.mkdir(parents=True, exist_ok=True)
        self.trials_df = pd.read_csv(self.csv_path)
        # Normalise column names just in case
        self.trials_df.columns = [c.strip() for c in self.trials_df.columns]
        self.base_data_dir = data_folder

    # ── Public API ──────────────────────────────────────────────────────────

    def run(self) -> None:
        """Loop over trials and epoch-units, honouring resume logic."""
        for row_idx, trial_row in self.trials_df.iterrows():
            trial_id  = int(trial_row["Trial"])  # e.g. 1 ➜ folder Trial_01
            trial_dir = self._prepare_trial_folder(trial_id)

            # ── Determine starting point ---------------------------------------------------
            ckpt_dir = trial_dir / "checkpoints"
            ckpt_dir.mkdir(exist_ok=True)
            start_unit = 0
            model = None
            if self.resume_flag:
                latest_ckpt = latest_checkpoint(ckpt_dir)
                if latest_ckpt is not None:
                    start_unit = parse_epoch_from_ckpt(latest_ckpt) + 1
                    model = models.load_model(latest_ckpt, compile=False)

            # ── Build model from scratch if needed ----------------------------------------
            if model is None:
                model = self._build_model(trial_row)

            # ── Training loop over *epoch units* -----------------------------------------
            for unit in range(start_unit, EPOCH_UNITS_PER_TRIAL):
                print(f"\nTrial {trial_id:02d}  •  Epoch-Unit {unit+1}/{EPOCH_UNITS_PER_TRIAL}")

                train_ds, val_ds = self._load_datasets(trial_row)
                
                '''#################### GOTTA UPDATE THIS FOR CONFIG ##############'''
                callbacks = self._make_callbacks(trial_row, unit)
                lr_state_path = experiment_folder / LR_STATE_SAVE_SUBDIR / LR_STATE_SAVE_FILENAME
                if Path(lr_state_path).exists:
                    Custom_Callbacks.reduce_lr_cb.load_state_from_file(lr_state_path)

                
                history = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    epochs=EPOCH_UNITS_PER_TRIAL,
                    steps_per_epoch=STEPS_PER_EPOCH_UNIT,
                    callbacks=callbacks,
                )

                # Overwrite history JSON each unit (requirement)
                Helper_Functions.save_history_to_json(
                    history,
                    metadata="history",  # single file always named history_*.json
                    save_path=str(trial_dir),
                )

            # ── End-of-trial artifacts ----------------------------------------------------
            self._finalise_trial(model, trial_dir)

        print("\n✅ All trials complete!")

    # ── Internals ───────────────────────────────────────────────────────────

    def _prepare_trial_folder(self, trial_num: int) -> Path:
        trial_dir = self.experiment_dir / f"Trial_{trial_num:02d}"
        trial_dir.mkdir(parents=True, exist_ok=True)
        (trial_dir / "checkpoints").mkdir(exist_ok=True)
        return trial_dir

    
    def _dataset_path_from_row(self, row: pd.Series, split: str, base_data_dir: str) -> str:
        """
        Given a CSV row and a split ('train', 'val', 'test'),
        return the full path to the TFRecord.
        """
        units_flag     = int(row["Size"])                       # 1-10
        style_flag     = str(row["Style"]).upper()[0]           # 'M' | 'I'
        smoothing_flag = (
            "C" if str(row["Smoothing"]).upper().startswith("C") else "B"
        )

        # shift descriptor intentionally ignored
        return build_dataset_filepath(
            split          = split,
            units          = units_flag,
            style_flag     = style_flag,
            smoothing_flag = smoothing_flag,
            base_data_dir  = base_data_dir
        )

    def _load_datasets(self, row: pd.Series, test: bool = False):
        """
        Instantiate *separate* train / validation datasets based on the CSV row.
        Also works as a train-test-set loader, set test = True when using _load_datasets
        test dataset pretends to be the val set when test = True
        """
        train_path = self._dataset_path_from_row(row, split="train")
        if test:
            val_path = self._dataset_path_from_row(row, split="test")
        else:
            val_path = self._dataset_path_from_row(row, split="val")

        train_ds = Data_Functions.prep_dataset_from_tfrecord(
            train_path,
            batch_size   = self.batch_size,
            shuffled     = True,                       # shuffle only training
            cut_background = not bool(row["Background"]),
        )
        train_ds = train_ds.repeat()

        val_ds = Data_Functions.prep_dataset_from_tfrecord(
            val_path,
            batch_size   = self.batch_size,
            shuffled     = False,                      # never shuffle validation
            cut_background = not bool(row["Background"]),
        )
        return train_ds, val_ds

        ########################################################################################
        ########################################################################################
    def _build_model(self, row: pd.Series) -> Model:
        """Create and compile a new model according to hyper-params in *row*."""
        # 🔲 TODO - Make compatible with config and save a copy of config
        dilation_mult = float(row.get("Dilation", 1.0))
        use_attention = bool(row.get("Attention", False))

        model = Custom_Models.create_modular_dcnn_model(
            dilation_multiplier=dilation_mult,
            use_local_attention=use_attention,
            use_long_range_attention=use_attention,
            use_final_attention=use_attention,
        )

        loss_fn = self._select_loss(row)
        metrics = Custom_Metrics.METRICS
        model.compile(
            optimizer=optimizers.Adam(),
            loss=loss_fn,
            metrics=metrics,
        )
        return model


    def _select_loss(self, row: pd.Series):
        """
        Choose and instantiate the loss for this trial, based on:
        • early_rewarding → wrap in Switching*Loss
        • smoothing       → “Custom” → focal, else → cross‐entropy
        • background      → background_removed = not background
        Expects columns:
        "Early Rewarding" (bool), "Smoothing" (str), "Background" (bool)
        """
        # 1) read your trial flags
        early_rewarding = bool(row["Early Rewarding"])
        smoothing      = str(row["Smoothing"]).lower()   # e.g. "custom" or "proper" or "none"
        background     = bool(row["Background"])
        background_removed = not background

        # 2) build the shared kwargs
        loss_kwargs = {
            "dominant_class_index":               DOMINANT_CLASS_INDEX,
            "dominant_correct_multiplier":        DOMINANT_CORRECT_MULTIPLIER,
            "dominant_incorrect_multiplier":      DOMINANT_INCORRECT_MULTIPLIER,
            "other_class_true_positive_multiplier":   OTHER_TP_MULTIPLIER,
            "other_class_false_negative_multiplier":  OTHER_FN_MULTIPLIER,
            "other_class_false_positive_multiplier":  OTHER_FP_MULTIPLIER,
            "other_class_true_negative_multiplier":   OTHER_TN_MULTIPLIER,
            "background_removed":                 background_removed,
            "threshold":                          THRESHOLD,
            "focal_gamma":                        FOCAL_GAMMA,
            "focal_alpha":                        FOCAL_ALPHA,
        }

        # 3) pick focal vs cross-entropy and add their hyper-params
        if smoothing == "custom":
            BaseLoss = Custom_Losses.CustomBinaryFocalLoss
            loss_kwargs.update({
                "smoothing_multiplier": INCORRECT_SMOOTHING_MULTIPLIER,
                "smoothing_as_correct":  DEFAULT_SMOOTHING_AS_CORRECT,
            })
        else:
            BaseLoss = Custom_Losses. AllBinaryFocalLoss
            if smoothing == "proper":
                loss_kwargs.update({
                    "label_smoothing": LABEL_SMOOTHING,
                })
            else:
                loss_kwargs.update({
                    "label_smoothing": 0,
                })

        # 4) if early_rewarding, wrap in the switching‐loss class
        if early_rewarding:
            loss_kwargs["swap_epoch"] = SWAP_EPOCH
            if BaseLoss is Custom_Losses.CustomBinaryFocalLoss:
                LossClass = Custom_Losses.SwitchingFocalLoss
            else:
                LossClass = Custom_Losses.SwitchingBinaryCrossentropyLoss
        else:
            LossClass = BaseLoss

        # 5) return the instantiated loss
        return LossClass(**loss_kwargs)


    def _make_callbacks(
        self,
        row: pd.Series,
        epoch_unit_idx: int,
    ) -> List[callbacks.Callback]:
        """Create per-epoch-unit callback list."""
        cbs_from_config = Custom_Callbacks.CALLBACKS

        tensorboard_dir = LOG_DIR + f"/{self.experiment}/trial_{row['Trial']:02d}/unit_{epoch_unit_idx:02d}"
        tb_cb = callbacks.TensorBoard(log_dir=tensorboard_dir, histogram_freq=1)

        cbs_from_config.append(tb_cb)
        return cbs_from_config

        ####################################################################################
        ####################################################################################
        # Breadcrumbs I aint dealing with tonight: switchable losses config: does it need **shared_kwargs 
        # Class EpochUpdater needs a pointer to a loss object so it maybe needs to get added in the training loop
    def _finalise_trial(self, model: Model, trial_dir: Path) -> None:
        """Plot train/val curves & save a compact *production* checkpoint."""
        # 🔲 TODO - capture the accumulated History over 10 units (requires persistence).
        history_json = trial_dir + "history_training_history.json"
        if history_json.exists():
            with open(history_json) as fh:
                history_dict = json.load(fh)
            # Use helper to generate plot and save
            fig = Helper_Functions.plot_train_val_curve(history_object=type("H", (), {"history": history_dict})(),
                                                        training_target_variable="loss")
            fig.savefig(trial_dir + "train_val_curve.jpg")

        # Save final model (weights only)
        model.save(trial_dir + "final_model.keras")


# ── Command-line helper ───────────────────────────────────────────────────────
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Run an Intron-Exon experiment from a CSV spec.")
    parser.add_argument("csv", help="Path to experiment CSV file")
    parser.add_argument("name", help="Experiment folder name (under Models/)")
    parser.add_argument("--no-resume", action="store_true", help="Start from scratch, ignore existing checkpoints")
    parser.add_argument("--batch", type=int, default=DEFAULT_BATCH_SIZE, help="Batch size")
    args = parser.parse_args()

    handler = ExperimentHandler(
        csv_path=args.csv,
        experiment_name=args.name,
        resume=not args.no_resume,
        batch_size=args.batch,
    )
    handler.run()
    
    # Move logs to experiment folder

┌────────────────────────────────────────────────────────────────────────┐
│  External call                                                        │
│  best_hp, tuner = run_tuning(data_spec, project_name="exp01")         │
└────────────┬──────────────────────────────────────────────────────────┘
             │
             ▼
┌────────────────────────────────────────────────────────────────────────┐
│  run_tuning()                                                         │
│  ├─ _prepare_datasets()         →  Data_Functions.prep_dataset…       │
│  │                                returns (train_ds, val_ds)          │
│  ├─ DCNNHyperModel(data_spec)                                         │
│  │    │                                                               │
│  │    └─ build(hp)                                                    │
│  │         ├─ hp.* search-space (lr, dilation_mult, attention …)      │
│  │         ├─ Custom_Models.create_modular_dcnn_model()               │
│  │         └─ model.compile( Custom_Losses.CustomBinaryFocalLoss , metrics )        │
│  ├─ kt.Hyperband( … directory=Models/<proj>/tuner/ )                  │
│  ├─ tuner.search(train_ds, val_ds, callbacks=[EarlyStopping])         │
│  ├─ best_hp = tuner.get_best_hyperparameters(1)[0]                    │
│  └─ _save_best(best_hp, project_name) → best_hparams.json             │
└────────────────────────────────────────────────────────────────────────┘

   ▲ load_best_hparams(project_name)  ←────────────┘ (JSON helper)


In [ ]:
# Hyperparameter_Tuning.py
"""
Keras-Tuner wrapper for the Intron-Exon project.

Usage
-----
>>> from IEModules import Hyperparameter_Tuning as hpt
>>> best_hp, tuner = hpt.run_tuning(
...     data_spec=row_from_csv,
...     project_name="exp01_hyperband"
... )
"""

from __future__ import annotations
import json, os, datetime
from pathlib import Path
from typing import Dict, Tuple

import keras_tuner as kt
import tensorflow as tf

from . import (
    Custom_Models,
    Custom_Losses,
    Data_Functions,
    Custom_Metrics,
    config
)

# ──────────────────────────────────────────────────────────────────────────────
# 1. HyperModel definition
# ──────────────────────────────────────────────────────────────────────────────
class DCNNHyperModel(kt.HyperModel):
    """Wraps create_modular_dcnn_model with a tunable search-space."""
    def __init__(self, data_spec: Dict):
        self.data_spec = data_spec          # row from your experiment CSV

    # --- mandatory -----------------------------------------------------------
    def build(self, hp: kt.HyperParameters) -> Model:
        # Search-space examples — extend freely.
        dilation_mult = hp.Float("dilation_mult", 0.5, 2.0, step=0.25)
        use_attention = hp.Boolean("attention", default=True)
        lr            = hp.Float("lr", 1e-4, 3e-3, sampling="log")

        model = Custom_Models.create_modular_dcnn_model(
            dilation_multiplier=dilation_mult,
            use_local_attention=use_attention,
            use_long_range_attention=use_attention,
            use_final_attention=use_attention,
        )

        model.compile(
            optimizer=optimizers.Adam(learning_rate=lr),
            loss      = Custom_Losses.Custom_Losses.CustomBinaryFocalLoss(),
            metrics   = [Custom_Metrics.CustomNoBackgroundF1Score(num_classes=5)],
        )
        return model

    # --- optional ------------------------------------------------------------
    def fit(self, hp, model, *args, **kwargs):
        """You can override to inject custom callbacks—or just omit."""
        return super().fit(hp, model, *args, **kwargs)

# ──────────────────────────────────────────────────────────────────────────────
# 2. Helper: get train/val datasets exactly once per tuner process
# ──────────────────────────────────────────────────────────────────────────────
def _prepare_datasets(data_spec: Dict, batch_size=32):
    tfrecord_path = Path(config.DATA_DIR) / "YOUR_DEFAULT.tfrecord.gz"
    # TODO - replace -- use data_spec to choose correct file
    ds = Data_Functions.prep_dataset_from_tfrecord(
        tfrecord_path,
        batch_size=batch_size,
        shuffled=True,
        cut_background=not bool(data_spec["Background"])
    )
    # simplistic 80-20 split
    n = ds.cardinality().numpy()
    train_ds = ds.take(int(n*0.8))
    val_ds   = ds.skip(int(n*0.8))
    return train_ds, val_ds

# ──────────────────────────────────────────────────────────────────────────────
# 3. Public API
# ──────────────────────────────────────────────────────────────────────────────
def run_tuning(
    data_spec: Dict,
    project_name: str,
    max_trials: int = 30,
    overwrite: bool = False,
) -> Tuple[kt.HyperParameters, kt.Tuner]:

    # Materialise datasets *once* so every trial re-uses the same TF graph
    train_ds, val_ds = _prepare_datasets(data_spec)

    hypermodel = DCNNHyperModel(data_spec)

    tuner = kt.Hyperband(
        hypermodel,
        objective   = kt.Objective("val_no_background_f1", direction="max"),
        max_epochs  = 15,
        factor      = 3,
        directory   = str(Path(config.MODEL_DIR) / project_name),
        project_name= "tuner",
        overwrite   = overwrite
    )

    stop_early = callbacks.EarlyStopping(
        monitor="val_no_background_f1",
        mode="max",
        patience=5,
        restore_best_weights=True
    )

    tuner.search(
        train_ds,
        validation_data = val_ds,
        callbacks       = [stop_early],
    )

    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    _save_best(best_hp, project_name)

    return best_hp, tuner

# ──────────────────────────────────────────────────────────────────────────────
# 4. Convenience helpers
# ──────────────────────────────────────────────────────────────────────────────
def _save_best(best_hp: kt.HyperParameters, project_name: str):
    out = Path(config.MODEL_DIR) / project_name / "best_hparams.json"
    with open(out, "w") as fh:
        json.dump(best_hp.values, fh, indent=2)
    print(f"Best HP written to {out}")

def load_best_hparams(project_name: str) -> Dict:
    path = Path(config.MODEL_DIR) / project_name / "best_hparams.json"
    with open(path) as fh:
        return json.load(fh)


In [ ]:
# anywhere - CLI, notebook, or even inside ExperimentHandler
from IEModules.Hyperparameter_Tuning import run_tuning, load_best_hparams

best_hp, tuner = run_tuning(
    data_spec   = row_from_csv_as_dict,
    project_name= "exp01_hyperband"
)
print("Best LR  :", best_hp['lr'])
print("Dilation :", best_hp['dilation_mult'])


In [ ]:
# experiment_framework.py
"""
High-level experiment orchestration for the Intron-Exon project.

Each CSV row represents a *trial* that is trained for a fixed number of
**epoch-units** (``config.EPOCH_UNITS_PER_TRIAL``).  One epoch-unit corresponds
exactly to ``config.STEPS_PER_EPOCH_UNIT`` gradient-descent steps on the
training dataset.

Responsibilities
----------------
*     Experiment folder setup and per-trial sub-folders
*     Incremental/resumable training with checkpoint discovery
*     Loading TFRecord datasets based on naming conventions
*     Selecting the appropriate model architecture, loss, metrics and callbacks
*     Aggregating Keras ``History`` objects across epoch-units and persisting
      them crash-safely as JSON
*     Re-plotting a train/val loss curve and saving a final model artefact at
      the end of every trial

Revision history
----------------
🔄 **2025-04-29 rewrite** —⁠ consolidated fixes discussed with ChatGPT:
    • Added missing imports, Path-safe ops, and metrics instantiation
    • Numeric checkpoint ordering
    • One-file running history with crash-safety
    • Correct ``epochs``/``initial_epoch`` per unit
    • Avoid mutating global callback list; TB/Updater added per unit
    • LR-scheduler state kept *per-trial*
    • ``_dataset_path_from_row`` uses ``self.base_data_dir``
    • Garbage-collect & clear Keras session between trials
"""
from __future__ import annotations

import json
import re
import tempfile
import shutil
import gc
import copy
from collections import defaultdict
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import pandas as pd
import tensorflow as tf
from keras import callbacks, optimizers, models, backend, utils

# ── In-house modules ───────────────────────────────────────────────────────────
from IEModules import (
    Data_Functions,
    Custom_Models,
    Custom_Callbacks,
    Custom_Losses,
    Helper_Functions,
    Custom_Metrics,
)
from IEModules.config import (
    MODEL_DIR,
    LOG_DIR,
    EPOCH_UNITS_PER_TRIAL,
    DEFAULT_BATCH_SIZE,
    STEPS_PER_EPOCH_UNIT,
    experiment_data_folder,
    LR_STATE_SAVE_SUBDIR,
    LR_STATE_SAVE_FILENAME,
)
from IEModules import config as cfg  # Generic access to loss hyper-params

# ── Dataset naming helpers ─────────────────────────────────────────────────────

def build_dataset_filepath(
    split: str,
    units: int,
    style_flag: str,
    smoothing_flag: str,
    base_data_dir: Path = experiment_data_folder,
) -> Path:
    """Resolve the unique TFRecord file that matches a naming convention.

    Parameters
    ----------
    split : str
        One of ``{"train", "val", "test"}``.
    units : int
        Size flag taken from the CSV (number of epoch-units included when the
        TFRecord was generated).
    style_flag : str
        Either ``"M"`` (mixed) or ``"I"`` (incremental).  Only the first
        character is used.
    smoothing_flag : str
        Either ``"C"`` (custom smoothed labels) or ``"B"`` (binary labels).
    base_data_dir : pathlib.Path, optional
        Directory that contains the dataset files.  Defaults to the folder
        chosen in *config.experiment_data_folder*.

    Returns
    -------
    pathlib.Path
        Fully-qualified path to the single matching TFRecord file.

    Raises
    ------
    FileNotFoundError
        If no file with the expected prefix exists.
    RuntimeError
        If more than one file matches the prefix (ambiguous).
    """
    split = split.lower()
    prefix = f"{split}_{units:02d}_{style_flag.upper()}_{smoothing_flag.upper()}"
    pattern = f"{prefix}*IEData.tfrecord.gz"
    matches = sorted(base_data_dir.glob(pattern))
    if not matches:
        raise FileNotFoundError(
            f"No TFRecord starting with '{prefix}' in {base_data_dir}")
    if len(matches) > 1:
        raise RuntimeError(f"Ambiguous prefix '{prefix}': {matches}")
    return matches[0]


# ── Utility helpers ────────────────────────────────────────────────────────────

def latest_checkpoint(ckpt_dir: Path) -> Optional[Path]:
    """Return the checkpoint with the highest *numeric* epoch identifier.

    Keras' ``ModelCheckpoint`` callback embeds the epoch number into the file
    name (e.g. ``epoch-012-val_*.keras``).  This helper extracts the number and
    returns the latest file so that training can resume from there.

    Parameters
    ----------
    ckpt_dir : pathlib.Path
        Directory that contains ``*.keras`` model files.

    Returns
    -------
    pathlib.Path | None
        Path to the checkpoint or ``None`` if the directory is empty.
    """
    epochs: Dict[int, Path] = {}
    for p in ckpt_dir.glob("*.keras"):
        m = re.search(r"epoch[-_](\d+)", p.name)
        if m:
            epochs[int(m.group(1))] = p
    return epochs[max(epochs)] if epochs else None


def _load_running_history(path: Path) -> Dict[str, List[float]]:
    """Load a *running* history JSON file if it exists else return an empty dict."""
    if path.exists():
        with open(path) as fh:
            return json.load(fh)
    return defaultdict(list)


def _extend_history(store: Dict[str, List], fragment: Dict[str, List]):
    """Append values from *fragment* into the list-of-lists *store* in-place."""
    for k, v in fragment.items():
        store.setdefault(k, []).extend(v)


def _atomic_dump(obj: Dict, path: Path):
    """Write *obj* as JSON to *path* atomically (write-then-move)."""
    tmp = tempfile.NamedTemporaryFile("w", delete=False)
    try:
        json.dump(obj, tmp)
        tmp.close()
        shutil.move(tmp.name, path)
    finally:
        if Path(tmp.name).exists():
            Path(tmp.name).unlink(missing_ok=True)


# ── Main handler ───────────────────────────────────────────────────────────────

class ExperimentHandler:
    """Orchestrate a *multi-trial* experiment defined by a CSV specification.

    Each trial is trained for ``config.EPOCH_UNITS_PER_TRIAL`` epoch-units.  The
    handler creates a dedicated sub-folder per trial, handles resumption logic
    (checkpoint + ReduceLROnPlateau state + aggregated history) and finally
    stores a *final* model plus a JPEG learning-curve.
    """

    # ────────────────────────────────────────────────────────────────────
    def __init__(
        self,
        csv_path: Path | str,
        experiment_name: str,
        model_dir: Path | str = MODEL_DIR,
        data_folder: Path | str = experiment_data_folder,
        resume: bool = True,
        batch_size: int = DEFAULT_BATCH_SIZE,
    ) -> None:
        """Construct a new *ExperimentHandler*.

        Parameters
        ----------
        csv_path : str | pathlib.Path
            Path to the CSV file whose rows specify the hyper-parameters for each
            trial (columns must match the expected headers).
        experiment_name : str
            Name of the experiment folder to be created inside ``MODEL_DIR``.
        model_dir : str | pathlib.Path, optional
            Root directory that will hold the *experiment* folder (default:
            ``config.MODEL_DIR``).
        data_folder : str | pathlib.Path, optional
            Directory that contains pre-built TFRecord datasets (default:
            ``config.experiment_data_folder``).
        resume : bool, optional
            If ``True`` (default) the handler will look for an existing
            checkpoint & aggregated history and continue training; otherwise it
            always starts fresh.
        batch_size : int, optional
            Mini-batch size fed to ``tf.data`` pipelines (default:
            ``config.DEFAULT_BATCH_SIZE``).
        """
        self.csv_path = Path(csv_path)
        self.experiment = experiment_name
        self.batch_size = batch_size
        self.resume_flag = resume
        self.experiment_dir = Path(model_dir) / experiment_name
        self.experiment_dir.mkdir(parents=True, exist_ok=True)
        self.trials_df = pd.read_csv(self.csv_path)
        self.trials_df.columns = [c.strip() for c in self.trials_df.columns]
        self.base_data_dir = Path(data_folder) / Path("TestValTrain")

    # ────────────────────────────────────────────────────────────────────
    def run(self):
        """Iterate over CSV rows and execute all trials sequentially."""
        for _, row in self.trials_df.iterrows():
            trial_id = int(row["Trial"])
            trial_dir = self._prepare_trial_folder(trial_id)
            history_path = trial_dir / "history_full.json"
            running_hist = _load_running_history(history_path)
            start_unit = len(next(iter(running_hist.values()), []))

            # ── Resume / build model ────────────────────────────────
            ckpt_dir = trial_dir / "checkpoints"
            model = None
            if self.resume_flag and (ckpt := latest_checkpoint(ckpt_dir)):
                custom_objs = utils.get_custom_objects()
                model = models.load_model(ckpt, compile=False, custom_objects=custom_objs)
            if model is None:
                backend.clear_session(); gc.collect()
                model = self._build_model(row)
            else:                                   # resume
                loss_fn = self._select_loss(row)
                metrics = [
                    m(num_classes=5) if "num_classes" in m.__init__.__code__.co_varnames else m()
                    for m in Custom_Metrics.METRICS
                ]                    
                model.compile(optimizer=optimizers.Adam(),
                            loss=loss_fn,
                            metrics=metrics)

            # ── Loop over epoch-units ───────────────────────────────
            for unit in range(start_unit, EPOCH_UNITS_PER_TRIAL):
                print(f"\nTrial {trial_id:02d} • Epoch-Unit {unit+1}/{EPOCH_UNITS_PER_TRIAL}")

                train_ds, val_ds = self._load_datasets(row)
                cbs = self._make_callbacks(row, unit, trial_dir, model)

                history = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    epochs=unit + 1,
                    initial_epoch=unit,
                    steps_per_epoch=STEPS_PER_EPOCH_UNIT,
                    callbacks=cbs,
                )

                _extend_history(running_hist, history.history)
                _atomic_dump(running_hist, history_path)

            # ── End-of-trial clean-up ───────────────────────────────
            Helper_Functions.plot_train_val_curve(
                history_object=type("H", (), {"history": running_hist})(),
                training_target_variable="loss",
            ).savefig(trial_dir / "train_val_curve.jpg")
            model.save(trial_dir / "final_model.keras")

        print("\n✅ All trials complete!")

    # ────────────────────────────────────────────────────────────────────
    def _prepare_trial_folder(self, num: int) -> Path:
        """Ensure the directory structure for a given trial exists.

        Creates ``Trial_XX/checkpoints`` (nested under the experiment folder)
        and returns the trial directory path.
        """
        tdir = self.experiment_dir / f"Trial_{num:02d}"
        (tdir / "checkpoints").mkdir(parents=True, exist_ok=True)
        return tdir

    # ---------------- Dataset helpers ----------------------------------
    def _dataset_path_from_row(self, row: pd.Series, split: str) -> Path:
        """Resolve the TFRecord file for *split* according to CSV flags."""
        units_flag = int(row["Size"])
        style_flag = str(row["Style"]).upper()[0]
        smoothing_flag = "C" if str(row["Smoothing"]).upper().startswith("C") else "B"
        return build_dataset_filepath(
            split,
            units_flag,
            style_flag,
            smoothing_flag,
            base_data_dir=self.base_data_dir,
        )

    def _load_datasets(self, row: pd.Series, test: bool = False):
        """Build ``tf.data`` datasets for training/validation.

        Parameters
        ----------
        row : pandas.Series
            The current CSV row describing the trial.
        test : bool, optional
            If ``True`` the *validation* dataset is taken from the ``test``
            TFRecord instead of ``val`` (useful for final evaluation).
        """
        train_path = self._dataset_path_from_row(row, "train")
        val_path = self._dataset_path_from_row(row, "test" if test else "val")
        train_ds = Data_Functions.prep_dataset_from_tfrecord(
            train_path,
            batch_size=self.batch_size,
            shuffled=True,
            cut_background=not bool(row["Background"]),
        ).repeat()
        val_ds = Data_Functions.prep_dataset_from_tfrecord(
            val_path,
            batch_size=self.batch_size,
            shuffled=False,
            cut_background=not bool(row["Background"]),
        )
        return train_ds, val_ds

    # ---------------- Model/loss building ------------------------------
    def _build_model(self, row: pd.Series):
        """Instantiate and compile a model based on CSV hyper-parameters."""
        dilation_mult = float(row.get("Dilation", 1.0))
        use_attention = bool(row.get("Attention", False))
        model = Custom_Models.create_modular_dcnn_model(
            dilation_multiplier=dilation_mult,
            use_local_attention=use_attention,
            use_long_range_attention=use_attention,
            use_final_attention=use_attention,
        )
        loss_fn = self._select_loss(row)
        metrics = [
            m(num_classes=5) if "num_classes" in m.__init__.__code__.co_varnames else m()
            for m in Custom_Metrics.METRICS
        ]
        model.compile(optimizer=optimizers.Adam(), loss=loss_fn, metrics=metrics)
        return model

    def _select_loss(self, row: pd.Series):
        """Return an appropriately parameterised loss object for the trial."""
        early_rewarding = bool(row["Early Rewarding"])
        smoothing = str(row["Smoothing"]).lower()
        background_removed = not bool(row["Background"])

        kwargs = dict(
            dominant_class_index=cfg.DOMINANT_CLASS_INDEX,
            dominant_correct_multiplier=cfg.DOMINANT_CORRECT_MULTIPLIER,
            dominant_incorrect_multiplier=cfg.DOMINANT_INCORRECT_MULTIPLIER,
            other_class_true_positive_multiplier=cfg.OTHER_TP_MULTIPLIER,
            other_class_false_negative_multiplier=cfg.OTHER_FN_MULTIPLIER,
            other_class_false_positive_multiplier=cfg.OTHER_FP_MULTIPLIER,
            other_class_true_negative_multiplier=cfg.OTHER_TN_MULTIPLIER,
            background_removed=background_removed,
            threshold=cfg.THRESHOLD,
            focal_gamma=cfg.FOCAL_GAMMA,
            focal_alpha=cfg.FOCAL_ALPHA,
        )

        if smoothing == "custom":
            BaseLoss = Custom_Losses.CustomBinaryFocalLoss
            kwargs.update(
                smoothing_multiplier=cfg.INCORRECT_SMOOTHING_MULTIPLIER,
                smoothing_as_correct=cfg.DEFAULT_SMOOTHING_AS_CORRECT,
            )
        else:
            BaseLoss = Custom_Losses. AllBinaryFocalLoss
            kwargs.update(label_smoothing=cfg.LABEL_SMOOTHING if smoothing == "proper" else 0)

        if early_rewarding:
            kwargs["swap_epoch"] = cfg.SWAP_EPOCH
            LossClass = (
                Custom_Losses.SwitchingFocalLoss
                if BaseLoss is Custom_Losses.CustomBinaryFocalLoss
                else Custom_Losses.SwitchingBinaryCrossentropyLoss
            )
        else:
            LossClass = BaseLoss
        return LossClass(**kwargs)

    # ---------------- Callbacks ----------------------------------------
    def _make_callbacks(
        self,
        row: pd.Series,
        unit_idx: int,
        trial_dir: Path,
        model,
    ) -> List[callbacks.Callback]:
        """Assemble a fresh list of callbacks for the *current* epoch-unit."""
        # Start with a shallow copy so that the global list is never mutated.
        cbs = [copy.deepcopy(cb) for cb in Custom_Callbacks.CALLBACKS]

        # --- TensorBoard ------------------------------------------------
        tb_dir = LOG_DIR / self.experiment / f"trial_{row['Trial']:02d}" / f"unit_{unit_idx:02d}"
        cbs.append(callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=1))

        # --- Epoch updater for switchable loss --------------------------
        if hasattr(model.loss, "epoch_var"):
            cbs.append(Custom_Losses.EpochUpdater(model.loss))

        # --- LR-scheduler state path (unique per trial) -----------------
        lr_state_path = trial_dir / LR_STATE_SAVE_SUBDIR / LR_STATE_SAVE_FILENAME
        for cb in cbs:
            if isinstance(cb, callbacks.ModelCheckpoint):
                cb.filepath = str(trial_dir / cfg.CHECKPOINT_SUBDIR / cfg.CHECKPOINT_FILENAME)
            if isinstance(cb, Custom_Callbacks.StatefulReduceLROnPlateau):
                cb.state_save_filepath = lr_state_path
                if lr_state_path.exists() and self.resume_flag and unit_idx == 0:
                    cb.load_state_from_file(lr_state_path)
        return cbs


# ── CLI helper ────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(
        description="Run an Intron-Exon experiment from a CSV spec.")
    parser.add_argument("csv", help="Path to experiment CSV file")
    parser.add_argument("name", help="Experiment folder name (under Models/)")
    parser.add_argument("--no-resume", action="store_true", help="Ignore existing checkpoints/history")
    parser.add_argument("--batch", type=int, default=DEFAULT_BATCH_SIZE, help="Batch size")
    args = parser.parse_args()

    handler = ExperimentHandler(
        csv_path=args.csv,
        experiment_name=args.name,
        resume=not args.no_resume,
        batch_size=args.batch,
    )
    handler.run()


2025-04-29 02:49:53.904750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-29 02:49:54.003581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-29 02:49:54.032874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-29 02:49:54.214451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-29 02:49:55.597796: W tensorflow/compiler/tf2

ModuleNotFoundError: No module named 'Custom_Callbacks'

In [5]:
# test_model_compile.py

import numpy as np
import tensorflow as tf
import copy

# 👉 adjust this import path if your packages live elsewhere
from IEModules.Custom_Models import create_modular_dcnn_model
from IEModules.Custom_Losses import (
    CustomBinaryFocalLoss,
    AllBinaryFocalLoss,
    SwitchingFocalLoss,
    SwitchingBinaryCrossentropyLoss,
)
from IEModules.Custom_Metrics import METRICS

def instantiate_metrics():
    """Deep-copy or re-instantiate your metric objects to avoid shared state."""
    out = [copy.deepcopy(m) for m in METRICS]
    return out

def main():
    # 1) Build the model
    model = create_modular_dcnn_model(
        input_dim=5,
        sequence_length=5000,
        num_classes=5,
        use_local_attention=False,    # toggle on/off to test
        use_long_range_attention=False,
        use_final_attention=False,
        dilation_multiplier=1.0,
    )

    # 2) Pick a loss (test both focal & crossentropy variants)
    loss_fn =  CustomBinaryFocalLoss()  # or CustomBinaryFocalLoss()

    # 3) Compile
    model.compile(
        optimizer=optimizers.Adam(),
        loss=loss_fn,
        metrics=instantiate_metrics(),
    )

    # 4) See the summary
    model.summary()

    # 5) Do a single train-on-batch with random data
    x_dummy = np.random.random((2, 5000, 4)).astype(np.float32)
    y_dummy = np.random.random((2, 5000, 4)).astype(np.float32)
    loss, *metric_vals = model.train_on_batch(x_dummy, y_dummy)
    print(f"\n✅ train_on_batch succeeded: loss={loss:.4f}, metrics={metric_vals}")

if __name__ == "__main__":
    main()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 5000, 5)   │          0 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 5000, 10)  │          0 │ input_layer_3[0]… │
│ (Concatenate)       │                   │            │ lambda_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_49 (Conv1D)  │ (None, 5000, 64)  │        704 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_50 (Conv1D)  │ (None, 5000, 64)  │     36,928 │ conv1d_49[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 64)  │        256 │ conv1d_50[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_34          │ (None, 5000, 64)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_51 (Conv1D)  │ (None, 5000, 64)  │     36,928 │ dropout_34[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 64)  │        256 │ conv1d_51[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_35          │ (None, 5000, 64)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 5000, 64)  │          0 │ dropout_35[0][0], │
│                     │                   │            │ conv1d_49[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_53 (Conv1D)  │ (None, 5000, 160) │     92,320 │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_53[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_36          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_54 (Conv1D)  │ (None, 5000, 160) │    230,560 │ dropout_36[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_54[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_37          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_52 (Conv1D)  │ (None, 5000, 160) │     10,400 │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, 5000, 160) │          0 │ dropout_37[0][0], │
│                     │                   │            │ conv1d_52[0][0] 

 Total params: 1,819,877 (6.94 MB)

 Trainable params: 1,816,805 (6.93 MB)

 Non-trainable params: 3,072 (12.00 KB)

ValueError: Input 0 of layer "functional_3" is incompatible with the layer: expected shape=(None, 5000, 5), found shape=(2, 5000, 4)

In [1]:
import time
import sys
import os
import glob
import math
import threading
import concurrent.futures as cf
import random
import re

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
from keras import backend as K
import gc
import keras_tuner as kt
from pyfaidx import Fasta

from IEModules.config import DATA_DIR, LOG_DIR, MODEL_DIR, MODULE_DIR, NOTEBOOK_DIR

@utils.register_keras_serializable()
def tile_to_batch(z):
    pe, x = z
    return tf.tile(pe, [tf.shape(x)[0], 1, 1])

@utils.register_keras_serializable()
class LocalMaskLayer(layers.Layer):
    def __init__(self, radius=3, **kwargs):
        """
        Args:
            radius (int): The allowed distance for positions (default 3).
        """
        super(LocalMaskLayer, self).__init__(**kwargs)
        self.radius = radius

    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        i = tf.range(seq_len)[:, None]
        j = tf.range(seq_len)[None, :]
        mask = tf.cast(tf.abs(i - j) <= self.radius, tf.float32)
        mask = tf.expand_dims(mask, 0)  # shape: (1, seq_len, seq_len)
        mask = tf.tile(mask, [tf.shape(inputs)[0], 1, 1])  # shape: (batch, seq_len, seq_len)
        return mask

    def get_config(self):
        config = super(LocalMaskLayer, self).get_config()
        config.update({'radius': self.radius})
        return config


# Helper fn to adjust dilation rates.
@utils.register_keras_serializable()
def adjust_dilation_rate(rate, dilation_multiplier):
    # Multiply by dilation_multiplier, round to nearest int, and make sure it is at least 1.
    new_rate = int(round(rate * dilation_multiplier))
    return new_rate if new_rate >= 1 else 1


@utils.register_keras_serializable()
def create_modular_dcnn_model(
    input_dim=5,
    sequence_length=5000,
    num_classes=5,
    use_local_attention=True,
    use_long_range_attention=True,
    use_final_attention=True,
    dilation_multiplier=1.0  # New parameter to scale the dilation rates
):
    

    inputs = Input(shape=(sequence_length, input_dim))
    
    # Condensed positional encoding block
    positions = tf.range(start=0, limit=sequence_length, delta=1)
    pos_encoding = layers.Embedding(input_dim=sequence_length, output_dim=num_classes)(positions)
    pos_encoding = tf.expand_dims(pos_encoding, axis=0)
    pos_encoding = layers.Lambda(tile_to_batch)([pos_encoding, inputs])
    
    concat_input = layers.Concatenate(axis=-1)([inputs, pos_encoding])
    
    # Option 1: Local Masked Attention after positional encoding
    if use_local_attention:
        local_mask = LocalMaskLayer()(concat_input)
        local_attn = layers.Attention(use_scale=True)([concat_input, concat_input], mask=[local_mask, local_mask])
        # # Residual connection
        concat_input = layers.Concatenate(axis=-1)([concat_input, local_attn])
    
    # Hyperparameters for dropout
    early_dropout = 0.1
    middle_dropout = 0
    late_dropout = 0.2

    cnn = layers.Conv1D(filters=64, kernel_size=9, activation='relu', padding='same')(concat_input)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(early_dropout)(cnn)
    
    # Dilated convolution block
    skip = concat_input
    skip = layers.Conv1D(filters=64, kernel_size=1, padding='same')(skip)
    dcnn = layers.Conv1D(filters=64, kernel_size=9, dilation_rate=adjust_dilation_rate(1, dilation_multiplier),  # originally 1
        activation='relu', padding='same')(skip)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(early_dropout)(dcnn)
    low_dcnn = dcnn
    
    dcnn = layers.Conv1D(filters=64, kernel_size=9, dilation_rate=adjust_dilation_rate(2, dilation_multiplier),  # originally 2
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(early_dropout)(dcnn)
    dcnn = layers.Add()([dcnn, skip])
    
    skip = dcnn
    skip = layers.Conv1D(filters=160, kernel_size=1, padding='same')(skip)
    dcnn = layers.Conv1D(filters=160, kernel_size=9, dilation_rate=adjust_dilation_rate(4, dilation_multiplier),  # originally 4
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(middle_dropout)(dcnn)
    
    dcnn = layers.Conv1D(filters=160, kernel_size=9, dilation_rate=adjust_dilation_rate(8, dilation_multiplier),  # originally 8
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(middle_dropout)(dcnn)
    dcnn = layers.Add()([dcnn, skip])
    
    skip = dcnn
    skip = layers.Conv1D(filters=192, kernel_size=1, padding='same')(skip)
    dcnn = layers.Conv1D(filters=192, kernel_size=9, dilation_rate=adjust_dilation_rate(16, dilation_multiplier),  # originally 16
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(middle_dropout)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=9, dilation_rate=adjust_dilation_rate(32, dilation_multiplier),  # originally 32
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(middle_dropout)(dcnn)
    dcnn = layers.Add()([dcnn, skip])
    
    skip = dcnn
    skip = layers.Conv1D(filters=192, kernel_size=1, padding='same')(skip)
    dcnn = layers.Conv1D(filters=192, kernel_size=9, dilation_rate=adjust_dilation_rate(64, dilation_multiplier),  # originally 64
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(middle_dropout)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=9, dilation_rate=adjust_dilation_rate(128, dilation_multiplier),  # originally 128
        activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(middle_dropout)(dcnn)
    dcnn = layers.Add()([dcnn, skip])
    
    # Concatenate inputs from different paths
    second_concat = layers.Concatenate(axis=-1)([concat_input, cnn, dcnn, low_dcnn])
    
    # Option 2: Long-range Attention after pooling the final convolution outputs
    if use_long_range_attention:
        pool_size = 10  # You can adjust the pooling factor as needed.
        pooled = layers.MaxPooling1D(pool_size=pool_size, padding='same')(second_concat)
        long_attn = layers.MultiHeadAttention(num_heads=4, key_dim=32, dropout=0.1)(pooled, pooled)
        long_attn_upsampled = layers.UpSampling1D(size=pool_size)(long_attn)
        second_concat = layers.Concatenate(axis=-1)([second_concat, long_attn_upsampled])
    
    # Option 3: Final Attention to capture which outputs are most important
    if use_final_attention:
        final_attn = layers.MultiHeadAttention(num_heads=4, key_dim=32, dropout=0.1)(second_concat, second_concat)
        second_concat = layers.Concatenate()([second_concat, final_attn])
    
    # Instead of flattening, use Conv1D (kernel_size=1) as dense layers.
    dense = layers.Conv1D(128, kernel_size=1, activation='relu')(second_concat)
    dense = layers.BatchNormalization()(dense)
    dense = layers.Dropout(late_dropout)(dense)
    
    dense = layers.Conv1D(128, kernel_size=1, activation='relu')(dense)
    dense = layers.BatchNormalization()(dense)
    dense = layers.Dropout(late_dropout)(dense)
    
    # Final classification layer (applied at every time step)
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='sigmoid')(dense)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

2025-04-29 20:45:17.681885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-29 20:45:17.744500: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-29 20:45:17.764353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-29 20:45:17.807802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-29 20:45:18.738792: W tensorflow/compiler/tf2

In [2]:
model = create_modular_dcnn_model()
model.summary(expand_nested=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 5000, 5)   │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 5000, 10)  │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ local_mask_layer    │ (None, 5000,      │          0 │ concatenate[0][0] │
│ (LocalMaskLayer)    │ 5000)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 5000, 10)  │          1 │ concatenate[0][0… │
│ (Attention)         │                   │            │ concatenate[0][0… │
│                     │                   │            │ local_mask_layer… │
│                     │                   │            │ local_mask_layer… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 5000, 20)  │          0 │ concatenate[0][0… │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 5000, 64)  │      1,344 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 5000, 64)  │     36,928 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 64)  │        256 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 5000, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 5000, 64)  │     36,928 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 64)  │        256 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 5000, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 5000, 64)  │          0 │ dropout_2[0][0],  │
│                     │                   │            │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 5000, 160) │     92,320 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 5000, 160) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 5000, 160) │    230,560 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_6[0][0]    │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 2,482,146 (9.47 MB)

 Trainable params: 2,479,074 (9.46 MB)

 Non-trainable params: 3,072 (12.00 KB)